In [ ]:
import json
import os
from Levenshtein import distance

Load expert selections and parsing results

In [ ]:
VALIDATION_DIRECTORY = '00_data/validation_data'

with open('{}/{}'.format(VALIDATION_DIRECTORY, 'validation_paragraphs.json'), encoding="utf-8") as f:
    val = json.load(f)

PARSED_DIRECTORY = '00_data/parsing_results'
PARSING_RESULTS = (file for file in os.listdir(PARSED_DIRECTORY) if (os.path.isfile(os.path.join(PARSED_DIRECTORY, file)) and file.endswith(".json")))

pars = {}
for file in PARSING_RESULTS:
    parsing_method = file.split('_')[0]
    company = file.split('_')[1]
    with open('{}/{}'.format(PARSED_DIRECTORY, file)) as f:
        parsed_text = json.load(f)
        if parsing_method not in pars:
            pars[parsing_method] = {}
        pars[parsing_method][company] = parsed_text

In [ ]:
parsing_method = 'pdfminer'
min_sentence_length = 15
max_dist_threshold = 0.05
relevant_sentences = {}

for company, topics in val.items():

    relevant_sentences[company] = {}
    report_text = pars[parsing_method][company]

    for topic, val_paragraphs in topics.items():
        relevant_sentences[company][topic] = []

        for val_paragraph in val_paragraphs:
            
            min_par_sim = 1
            for rep_par_idx, rep_par_sentences in report_text.items():
                for rep_par_sent_idx, rep_par_sent in rep_par_sentences.items():
                    min_sim = 1
                    if len(rep_par_sent) >= min_sentence_length:
                        start_idx = 0
                        end_idx = len(rep_par_sent)

                        while end_idx < len(val_paragraph):

                            test_str = val_paragraph[start_idx:end_idx]
                            dist = distance(rep_par_sent, test_str) / len(rep_par_sent)

                            if dist < min_sim:
                                min_sim = dist

                            start_idx += 1
                            end_idx += 1

                        if min_sim <= max_dist_threshold:
                            relevant_sentences[company][topic].append((rep_par_idx, rep_par_sent_idx))

with open('{}/{}_{}.json'.format(VALIDATION_DIRECTORY, parsing_method, 'matched_sentences'), 'w') as f:
    json.dump(relevant_sentences, f, indent=4)

In [ ]:
parsing_method = 'easyocr'
min_sentence_length = 15
max_dist_threshold = 0.1
relevant_sentences = {}

for company, topics in val.items():

    relevant_sentences[company] = {}
    report_text = pars[parsing_method][company]

    for topic, val_paragraphs in topics.items():
        relevant_sentences[company][topic] = []

        for val_paragraph in val_paragraphs:
            
            min_par_sim = 1
            for rep_par_idx, rep_par_sentences in report_text.items():
                for rep_par_sent_idx, rep_par_sent in rep_par_sentences.items():
                    min_sim = 1
                    if len(rep_par_sent) >= min_sentence_length:
                        start_idx = 0
                        end_idx = len(rep_par_sent)

                        while end_idx < len(val_paragraph):

                            test_str = val_paragraph[start_idx:end_idx]
                            dist = distance(rep_par_sent, test_str) / len(rep_par_sent)

                            if dist < min_sim:
                                min_sim = dist

                            start_idx += 1
                            end_idx += 1

                        if min_sim <= max_dist_threshold:
                            relevant_sentences[company][topic].append((rep_par_idx, rep_par_sent_idx))

with open('{}/{}_{}.json'.format(VALIDATION_DIRECTORY, parsing_method, 'matched_sentences'), 'w') as f:
    json.dump(relevant_sentences, f, indent=4)